In [1]:
from nd2reader import ND2Reader
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

import sys  
sys.path.insert(0, 'lib')
from iplabs import IPLabViewer as viewer

## SIM Input images

There are **120** raw images, each containing **3** channels. This results in a total of **360** different raw images that can be used as inputs. This number can be increased by a factor of two by rotating the images $180^{\circ}$, which leaves us with a **total of 720 input-output pairs**.

In [ ]:
sim_input = ND2Reader('DNN4SIM_data/data/raw/Image001/SIM_input.nd2')
print('Metadata:\n', sim_input.metadata)
print('\nSizes:\n', sim_input.sizes)
print('\nNumber of rows (angles): ', sim_input.sizes['y'] / 512)
print('Number of columns (phases): ', sim_input.sizes['x'] / 512)

In [ ]:
sim_inputs = {}
sim_input.iter_axes = 'c'
for i, channel in enumerate(sim_input):
    sim_inputs[sim_input.metadata['channels'][i]]= channel

plt.close('all')
view = viewer(list(sim_inputs.values()), title=list(sim_inputs.keys()), widgets=True)

## SIM Output images

In [ ]:
sim_output = ND2Reader('DNN4SIM_data/data/raw/Image001/SIM_output.nd2')
print('Metadata:\n', sim_output.metadata)
print('\nSizes:\n', sim_output.sizes)

In [ ]:
sim_outputs = {}
sim_output.iter_axes = 'c'
for i, channel in enumerate(sim_output):
    sim_outputs[sim_output.metadata['channels'][i]]= channel

plt.close('all')
view = viewer(list(sim_outputs.values()), title=list(sim_outputs.keys()), widgets=True)

## Input phases exploration

Which combination of phases and angles is best?

In [ ]:
phase_test = np.array(list(sim_inputs.values())[0])

phase_test = phase_test[:512, :]

phase_imgs = []
width = 512
for i in range(phase_test.shape[1] // width):
    if i in [1, 3]:
        phase_imgs.append(phase_test[:, i*width:(i+1)*width])
    
phase_imgs_avg = np.mean(phase_imgs, axis = 0)

plt.close('all')
view = viewer(phase_imgs_avg)

## Pre-Processing

The only change that preserves the angle of the Structured Illumination is a **rotation by $180^{\circ}$**. This means we can only increase the number of images by a factor of 2.

In [ ]:
pp_test = np.array(list(sim_inputs.values())[0])[:512,:512]

pps = [pp_test]
pps_names = ['orig']
for i in range(3):
    pps.append(np.rot90(pps[i]))
    pps_names.append(f'rot {int(90*(i+1))}')
for i in range(4):
    pps.append(np.flip(pps[i], axis = 0))
    pps_names.append(f'rot {int(90*i)} flipped')
    
plt.close('all')
view = viewer(pps, title=pps_names, subplots=(2, len(pps)//2))

## Calculating the illumination angle

In [ ]:
import numpy.fft as fft
import skimage

# Difference of Gaussian
def dog(img, sigma1):
    output = np.copy(img)
    sigma2 = np.sqrt(2)*sigma1
    filt1 = skimage.filters.gaussian(img, sigma = sigma1 , mode = 'reflect', truncate = 3, preserve_range = True)
    filt2 = skimage.filters.gaussian(img, sigma = sigma2 , mode = 'reflect', truncate = 3, preserve_range = True)
    output = filt1 - filt2
    # normalize output
    output = (output - np.min(output)) /(np.max(output) - np.min(output))
    return output

# Local max in a 3x3 nbh
def local_max(img, T):
    output = np.zeros(img.shape)
    coord = skimage.feature.peak_local_max(img, min_distance=1, threshold_rel = T)
    output[coord[:,0],coord[:,1]] = np.max(img)
    return output

In [ ]:
channel = 0
angle = 0
phase = 3
sig = 2
T = 0.7

img = np.array(list(sim_inputs.values())[channel])[angle*512:(angle+1)*512, phase*512:(phase+1)*512]

# FFT in dB
img_ft = 10 * np.log10(np.abs(fft.fftshift(fft.fft2(img))))

# Detect spots
img_ft_detect = dog(img_ft, sig)
coords = skimage.feature.peak_local_max(img_ft_detect, min_distance=1, threshold_rel = T)
print('Coords:\n', coords)

# Calculate angle
if coords.shape[0] == 3:
    print(f'\nAngle: {(360 * np.arctan2(coords[2,0] - coords[1,0], coords[2,1] - coords[1,1]) / (2*np.pi)):.3f} deg')

plt.close('all')
img_ft_detect = local_max(img_ft_detect, T)
view = viewer([img, img_ft, img_ft_detect], subplots=(1,3))

Illumination angles:
* $\alpha_1 = 13.815^{\circ}$
* $\alpha_2 = 133.698^{\circ}$
* $\alpha_3 = 254.181^{\circ}$

$\Delta \alpha \approx 120^{\circ}$

## Pipeline for importing the Dataset

In [7]:
from nd2reader import ND2Reader
import numpy as np
from pathlib import Path
# Print colors
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Load the sim_input image from 'path'
def load_sim_input(path, phases=[0,1,2,3,4], angles=[0,1,2]):
    sim_in = ND2Reader(path)
    sim_ch = []
    sim_in.iter_axes = 'c'
    for channel in sim_in:
        imgs = []
        for angle in angles:
            for phase in phases:
                new_img = channel[angle*512:(angle+1)*512, phase*512:(phase+1)*512]
                if len(imgs) > 0 and new_img.shape != imgs[-1].shape:
                    print(f'{bcolors.FAIL}Size problem with file {path}, skipping this file.{bcolors.ENDC}')
                    raise ValueError()
                imgs.append(new_img)    
        
        sim_ch.append(np.array(imgs))
        
    
    
    return sim_ch

# Load the sim_output image from 'path'
def load_sim_output(path):
    sim_in = ND2Reader(path)
    sim_ch = []
    sim_in.iter_axes = 'c'
    for channel in sim_in:
        if channel.shape != (1024,1024):
            print(f'{bcolors.FAIL}Size problem with file {path}, skipping this file.{bcolors.ENDC}')
            raise ValueError()
        
        sim_ch.append(np.array(channel))
    
    return sim_ch
    
# Load the entire sim dataset into a numpy array
def load_sim_dataset(path_to_raw, phases=[0,1,2,3,4], angles=[0,1,2]):
    ds = []
    ls = []
    path_to_raw = Path(path_to_raw)

    # Loop through all image folders and add the sim images to the dataset
    print('Collecting images...   0%', end='\r')
    subdir_list = [f for f in path_to_raw.glob('*')]
    for i, f in enumerate(subdir_list):
        # Input images
        input_path = f / 'SIM_input.nd2'
        try:
            # Append images to data
            ds = ds + load_sim_input(input_path, phases=phases, angles=angles)
        except (FileNotFoundError, ValueError) as err:
            if type(err) == FileNotFoundError:
                print(f'{bcolors.FAIL}File missing: {input_path}{bcolors.ENDC}')
                continue
            elif type(err) == ValueError:
                continue

        # Output images
        output_path = f / 'SIM_output.nd2'
        try:
            # Append images to labels
            ls = ls + load_sim_output(output_path)
        except (FileNotFoundError, ValueError) as err:
            if type(err) == FileNotFoundError:
                print(f'{bcolors.FAIL}File missing: {output_path}{bcolors.ENDC}')
                ds.pop(-1)
            elif type(err) == ValueError:
                ds.pop(-1)
        print(f'Collecting images... {i/len(subdir_list)*100:3.0f}%', end='\r')
    
    print(f'Collecting images... {bcolors.OKGREEN}100%{bcolors.ENDC}')
    # Check that data and labels have the same length
    if len(ds) != len(ls):
        raise ValueError('The data and labels are not of the same length.')
    
    # Create output dictionary
    output = {'data' : np.array(ds), 'labels' : np.array(ls)}
        
    return output

# Augment dataset (add a 180 degree rotated copy of the dataset and shuffle the result)
def augment_dataset(ds):
    skip = False
    if skip is False:
        # Normalize images
        print('Normalizing images...   0%', end='\r')
        # Calculate min an max for data and labels
        min_data = np.min(np.min(ds['data'], axis=-1), axis=-1)
        max_data = np.max(np.max(ds['data'], axis=-1), axis=-1)
        min_labels = np.min(np.min(ds['labels'], axis=-1), axis=-1)
        max_labels = np.max(np.max(ds['labels'], axis=-1), axis=-1)
        # Normalize all images to [0, 1]
        data_length = ds['data'].shape[0]
        ds_ = {'data' : ds['data'].copy().astype(np.float64), 'labels' : ds['labels'].copy().astype(np.float32)}
        for i in range(data_length):
            for j in range(ds['data'].shape[1]):
                ds_['data'][i][j] = (ds['data'][i][j].astype(np.float64) - min_data[i][j]) / (max_data[i][j] - min_data[i][j])
            ds_['labels'][i] = (ds['labels'][i].astype(np.float64) - min_labels[i]) / (max_labels[i] - min_labels[i])
            print(f'Normalizing images... {i/data_length*100:3.0f}%', end='\r')

        print(f'Normalizing images... {bcolors.OKGREEN}100%{bcolors.ENDC}')
    # Rotate images
    print('Augmenting dataset...', end='\r')
    # Create new dataset with rotated images and labels
    ds_['data'] = np.concatenate((ds_['data'], np.rot90(ds_['data'], k=2, axes=(2,3))))
    ds_['labels'] = np.concatenate((ds_['labels'], np.rot90(ds_['labels'], k=2, axes=(1,2))))
    print(f'Augmenting dataset... {bcolors.OKGREEN}Done{bcolors.ENDC}')
    # Shuffle the dataset
    print('Shuffling dataset...', end='\r')
    rng = np.random.default_rng()
    idx = rng.permutation(ds_['data'].shape[0])
    ds_['data'] = ds_['data'][idx]
    ds_['labels'] = ds_['labels'][idx]
    print(f'Shuffling dataset... {bcolors.OKGREEN}Done{bcolors.ENDC}')
    
    return ds_

In [3]:
#dataset = load_sim_dataset('DNN4SIM_data/data/raw')
dataset = load_sim_dataset('DNN4SIM_data/data/raw', phases=[1,3])

C:\Users\kay-1\Anaconda3\envs\DNN4SIM_env\lib\site-packages\nd2reader\raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


Size problem with file DNN4SIM_data\data\raw\Image029\SIM_input.nd2, skipping this file.
File missing: DNN4SIM_data\data\raw\Image056\SIM_input.nd2
File missing: DNN4SIM_data\data\raw\Image057\SIM_input.nd2
File missing: DNN4SIM_data\data\raw\Image094\SIM_input.nd2
Finished loading dataset.


In [8]:
aug_dataset = augment_dataset(dataset)

Normalizing images... 100%
Augmenting dataset... Done
Shuffling dataset... Done


In [9]:
print('Data: ', aug_dataset['data'].shape)
print('Labels: ', aug_dataset['labels'].shape)

# Show a random collection of 5 images and their corresponding labels
r_0 = np.random.randint(0, aug_dataset['data'].shape[0], 5)
r_1 = np.random.randint(0, aug_dataset['data'].shape[1], 5)
img_list = []
img_names = []
for i in range(len(r_0)):
    img_list.append(aug_dataset['data'][r_0[i]][r_1[i]])
    img_names.append('data')
for i in range(len(r_0)):
    img_list.append(aug_dataset['labels'][r_0[i]])
    img_names.append('label')

plt.close('all')
viewer(img_list, title=img_names, subplots=(2,5))

Data:  (696, 6, 512, 512)
Labels:  (696, 1024, 1024)


Button(description='Show Widgets', style=ButtonStyle())

In [11]:
np.unique(aug_dataset['data'][2][5])

array([0.00000000e+00, 8.69111768e-05, 2.60733530e-04, ...,
       9.03963150e-01, 9.67929776e-01, 1.00000000e+00])

In [28]:
import json

with open('Filtering.ipynb') as nb:
    d = json.load(nb)
    print(d['cells'][0].keys())

dict_keys(['cell_type', 'metadata', 'source'])
